In [1]:
import os
os.chdir('../movies')
from movieLens import MovieLens

In [2]:
ml = MovieLens()
ratings = ml.ratings.copy()
movies = ml.movies.copy()

In [3]:
import pandas  as pd

## Búsqueda de los usuarios

Para evaluar la efectividad de los diferentes algoritmos de recomendación, es importante considerar la variabilidad en los gustos y experiencias de los usuarios. En este sentido, se llevará a cabo un experimento utilizando dos usuarios distintos como casos de estudio. El objetivo es analizar cómo los algoritmos de recomendación se comportan al proporcionar recomendaciones personalizadas a cada uno de estos usuarios.

In [4]:
# Filter the ratings DataFrame for the target user ID
user_data_1 = ratings[ratings['userId'] == 1]
print(len(user_data_1))

# Print all information associated with the target user
# print(user_data_1)

movies_for_user_1 = user_data_1.groupby('userId')['movieId'].unique()
movies_for_user_1

232


userId
1    [1, 3, 6, 47, 50, 70, 101, 110, 151, 157, 163,...
Name: movieId, dtype: object

In [5]:
# Filter the ratings DataFrame for the target user ID
user_data_4 = ratings[ratings['userId'] == 4]
print(len(user_data_4))

# Print all information associated with the target user
# print(user_data_85)

movies_for_user_4 = user_data_4.groupby('userId')['movieId'].unique()
movies_for_user_4

216


userId
4    [21, 32, 45, 47, 52, 58, 106, 125, 126, 162, 1...
Name: movieId, dtype: object

In [6]:
# Convert the elements in the Series to strings
movies_for_user_1 = movies_for_user_1.astype(str)
movies_for_user_4 = movies_for_user_4.astype(str)

# Find the common elements in both series
common_elements = pd.Series(list(set(movies_for_user_1) & set(movies_for_user_4)), dtype=object)

# Print the common elements
print(common_elements)

Series([], dtype: object)


Para evaluar de manera efectiva los algoritmos de recomendación en un contexto específico, se ha realizado una selección cuidadosa de dos usuarios que han valorado más o menos el mismo números de películas pero que no han visto ninguna película en común (userId = 1 y userId = 4). Esta elección se ha llevado a cabo con el propósito de analizar cómo los algoritmos pueden generar recomendaciones personalizadas tomando como referencia dos usuarios que no comparten ninguna experiencia cinematográfica previa.

# Caracterización de los usuarios

In [7]:
# def getPreferences(userId, ratings, k, movieID_to_name):
#     """
#     Get the top k movies with the highest ratings and the bottom k movies with the lowest ratings for a given userId.

#     Args:
#         userId (int): The ID of the user.
#         ratings (pandas.DataFrame): The ratings dataset containing columns 'userId', 'movieId', and 'rating'.
#         k (int): The number of top and bottom movies to retrieve.
#         movieID_to_name (dict): A dictionary mapping movieId to movie title.

#     Returns:
#         top_movies (pandas.DataFrame): The DataFrame with the top k movies with highest ratings, including movie names.
#         bottom_movies (pandas.DataFrame): The DataFrame with the bottom k movies with lowest ratings, including movie names.
#     """

#     # Filter ratings for the specified userId
#     user_ratings = ratings[ratings['userId'] == userId].copy()
#     user_ratings['title'] = user_ratings['movieId'].map(movieID_to_name)
#     user_ratings = user_ratings[['userId','movieId','title','rating', ]]

#     # Sort movies by rating in descending order
#     sorted_movies = user_ratings.sort_values(by='rating', ascending=False)

#     # Get the top k movies with highest and lowest ratings
#     top_movies = sorted_movies.head(k).copy()
#     bottom_movies = sorted_movies.tail(k).copy()  

#     return top_movies, bottom_movies


In [8]:
# top_movies_user1, bottom_movies_user1 = getPreferences(1, ratings, 10, ml.movieID_to_name)
# top_movies_user1

Se utiliza la función getPreferences de la clase movieLens para obtener las principales preferencias de un usuario en cuanto a películas. Toma como entrada el ID de usuario y el número de películas a seleccionar. La función devuelve dos DataFrames: uno con las k películas con las calificaciones más altas y otro con las k películas con las calificaciones más bajas, incluyendo el título de cada película.

In [9]:
top_movies_user1, bottom_movies_user1 = ml.getPreferences(1,10)
top_movies_user1

,userId,movieId,title,rating,genres
231,1,5060,M*A*S*H (a.k.a. MASH),5.0,"[Comedy, Drama, War]"
185,1,2872,Excalibur,5.0,"[Adventure, Fantasy]"
89,1,1291,Indiana Jones and the Last Crusade,5.0,"[Action, Adventure]"
90,1,1298,Pink Floyd: The Wall,5.0,"[Drama, Musical]"
190,1,2948,From Russia with Love,5.0,"[Action, Adventure, Thriller]"
189,1,2947,Goldfinger,5.0,"[Action, Adventure, Thriller]"
188,1,2944,"Dirty Dozen, The",5.0,"[Action, Drama, War]"
186,1,2899,Gulliver's Travels,5.0,"[Adventure, Animation, Children]"
184,1,2858,American Beauty,5.0,"[Drama, Romance]"
179,1,2700,"South Park: Bigger, Longer and Uncut",5.0,"[Animation, Comedy, Musical]"


In [10]:
top_movies_user4, bottom_movies_user4 = ml.getPreferences(4,10)
bottom_movies_user4

,userId,movieId,title,rating,genres
167,4,3175,Galaxy Quest,1.0,"[Adventure, Comedy, Sci-Fi]"
95,4,1391,Mars Attacks!,1.0,"[Action, Comedy, Sci-Fi]"
34,4,509,"Piano, The",1.0,"[Drama, Romance]"
81,4,1211,"Wings of Desire (Himmel über Berlin, Der)",1.0,"[Drama, Fantasy, Romance]"
177,4,3489,Hook,1.0,"[Adventure, Comedy, Fantasy]"
142,4,2571,"Matrix, The",1.0,"[Action, Sci-Fi, Thriller]"
74,4,1183,"English Patient, The",1.0,"[Drama, Romance, War]"
189,4,4014,Chocolat,1.0,"[Drama, Romance]"
36,4,539,Sleepless in Seattle,1.0,"[Comedy, Drama, Romance]"
180,4,3591,Mr. Mom,1.0,"[Comedy, Drama]"


Una vez obtenidas las películas que más y menos les han gustado a los usuarios podemos obtener cuales son los géneros más y menos populares asociados a cada uno de los perfiles en base a las películas seleccionadas. **¿Tiene sentido?**